### 라이브러리, 데이터

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import matplotlib
import os
import glob

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# CNN 모델에 맞게 입력 이미지 형태 변경
x_image = tf.reshape(x, [-1, 28, 28, 1])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11ce09b00>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11c09e9b0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11c7364a8>)

### 모델 구축

In [8]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0, 1, shape=shape)
    return tf.Variable(initial)

In [10]:
# 합성곱, 패딩, 풀링 적용을 위한 함수 정의

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2X2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [11]:
# 첫번째 합성곱 계층의 가중치, 편향 정의

w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
w_conv1, b_conv1

(<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(32,) dtype=float32_ref>)

In [13]:
# ReLU 함수 및 풀링 계층 정의

h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
h_pool1 = max_pool_2X2(h_conv1)
h_conv1, h_pool1

(<tf.Tensor 'Relu:0' shape=(?, 28, 28, 32) dtype=float32>,
 <tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>)

In [ ]:
-

In [14]:
# 두번째 합성곱 계층의 가중치, 편향 정의

w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2X2(h_conv2)

In [15]:
# Fully connected net을 정의

w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

In [16]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# 드랍아웃 정의
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# softmax regression을 적용하여 최종 결과값을 정의
w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

### 모델 학습

In [18]:
# 모델 학습 및 테스트를 통해 정확도 출력

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [20]:
sess = tf.Session()

sess.run(tf.initialize_all_variables())

for i in range(2000):
    batch = mnist.train.next_batch(100)
    if i % 10 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, train accuracy %g" % (i, train_accuracy))
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, train accuracy 0.07
step 10, train accuracy 0.41
step 20, train accuracy 0.58
step 30, train accuracy 0.76
step 40, train accuracy 0.79
step 50, train accuracy 0.8
step 60, train accuracy 0.82
step 70, train accuracy 0.85
step 80, train accuracy 0.87
step 90, train accuracy 0.95
step 100, train accuracy 0.92
step 110, train accuracy 0.93
step 120, train accuracy 0.9
step 130, train accuracy 0.92
step 140, train accuracy 0.92
step 150, train accuracy 0.87
step 160, train accuracy 0.89
step 170, train accuracy 0.97
step 180, train accuracy 0.91
step 190, train accuracy 0.95
step 200, train accuracy 0.93
step 210, train accuracy 0.93
step 220, train accuracy 0.95
step 230, train accuracy 0.94
step 240, train accuracy 0.95
step 250, train accuracy 0.93
step 260, train accuracy 0.93
step 270, train accuracy 0.96
step 280, train accuracy 0.93
step 290, train accuracy 0.95
step 300, train accuracy 0.93
step 310, train accuracy 0.96
step 320, train accuracy 0.95
step 330, train accurac

In [22]:
print("test accuracy %g" % sess.run(accuracy, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob:1.0}))

test accuracy 0.982


### DNN - single layer

In [30]:
# 이미지와 라벨 값 정의

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# 가중치와 편향 정의

w = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 출력값 정의

y = tf.nn.softmax(tf.matmul(x, w) + b)

# 손실 함수 정의

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# 추정 방법(경사 하강법) 정의

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [31]:
# 모든 변수 초기화, 세션 시작

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [32]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [37]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [40]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9158


### DNN - multi layer

In [44]:
# weight & bias initialization

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0, 1, shape=shape)
    return tf.Variable(initial)

In [45]:
# 첫번째 은닉 계층의 가중치, 편향 정의
w_1 = weight_variable([784, 360])
b_1 = bias_variable([360])

# ReLU 함수 적용
h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

# 두번째 은닉 계층의 가중치, 편향 정의
w_2 = weight_variable([360, 180])
b_2 = bias_variable([180])

# ReLU 함수 정의
h_2 = tf.nn.relu(tf.matmul(h_1, w_2) + b_2)

# 세번째 은닉 계층의 가중치, 편향 정의
w_3 = weight_variable([180, 10])
b_3 = bias_variable([10])

# 소프트맥스 함수 적용
y = tf.nn.softmax(tf.matmul(h_2, w_3) + b_3)

In [48]:
# 손실 함수 및 추정방법, 기타 함수 정의

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 모든 변수 초기화, 세션 시작
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# 1000번의 반복 학습 후 최종 예측값 및 정확도 출력

for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9388
